## Preprocess
### Process toxin-antitoxin (TA) genomic sequence to prepare dataset for EVO fine-tuning
- Dataset: TADB 3.0 (https://bioinfo-mml.sjtu.edu.cn/TADB3/index.php)
- Toxin-Antitoxin type: Type II, Experimentally validated + in silico predicted

Evo: Fine-tuning requirements:
- MAX_seq_LENGTH = 1024 # Context length for finetuning
- Special characters:
    - '`': Toxin start
    - '!': Antitoxin start
    - '@': Type II TA pair

### Basic set up

Hyperparameters for Evo (8k)

In [ ]:
MAX_CONTEXT_LENGTH = 1024 # Context length for finetuning (including special tokens)

TA_SPECIAL_TOKEN = {
    'T': '`', # Toxin gene
    'AT': '!', # Antitoxin gene
    '2' : '@' # Type II Toxin-Antitoxin
}

File directories and other information

In [ ]:
# Input data paths
at_exp_path = '../raw_data/type_II_AT_exp_nucl.fas'
at_pre_path = '../raw_data/type_II_AT_pre_nucl.fas'
t_exp_path = '../raw_data/type_II_T_exp_nucl.fas'
t_pre_path = '../raw_data/type_II_T_pre_nucl.fas'

genome_csv_path_exp = '../data/NCBI_genome_exp.csv'

# Train dataset size
train_size_1k = 1000
train_size_5k = 5000
train_size_15k = 15000

# Output data path
genome_csv_path_1k = '../data/NCBI_genome_1k.csv'
json_output_path_1k = '../data/training_data_1k.json'

genome_csv_path_5k = '../data/NCBI_genome_5k.csv'
json_output_path_5k = '../data/training_data_5k_2.json'

genome_csv_path_15k = '../data/NCBI_genome_15k.csv'
json_output_path_15k = '../data/training_data_15k.json'

# Current output path
genome_csv_path = genome_csv_path_5k
json_output_path = json_output_path_5k

# NCBI Log in info
email_id = 'chang.m.yun@stanford.edu'

Import modules

In [ ]:
import argparse
from Bio import SeqIO, Entrez
import gzip
import json
import os
import pandas as pd
import re

### Read FASTA files

Open, parse, and store as pandas dataFrames

In [ ]:
# Functions
def parse_description(description=str):
    '''
    Parse description into 'Pair #', 'Accession', 'Start', 'End', 'Species'
    '''
    # Define the regular expression pattern
    pattern = r'^(\w+)\s+(\S+):c?(\d+)-(\d+)\s+\[(.*?)\]$'

    match = re.match(pattern, description)
    if match:
        # Extract information from matched groups
        code = match.group(1)
        accession = match.group(2)
        start = match.group(3)
        end = match.group(4)
        species = match.group(5)

        # Return extracted information
        return code, accession, start, end, species
    else:
        # Return None if no match found
        return None

def fasta_to_df(fasta_path=str, A_T=str, source=str, tat_type=str):
    '''
    Convert FASTA file into Pandas DataFrame
    '''
    # Define new pandas dataFrame: Source, Pair #, Accession #, Start position, End position, Sequence
    columns = ['type', 'source', 'pair_no', 'accession', 'start', 'end', 'seq']
    df = pd.DataFrame(columns=columns)

    # Open FASTA file
    with open(fasta_path, "r") as fasta_handle:
        fasta_seqio = SeqIO.parse(fasta_handle, 'fasta')

        # Populate FASTA into DataFrame
        for fasta in fasta_seqio:
            # Identify Header, Sequence, Description in FASTA
            header, seq, description = str(fasta.id), str(fasta.seq), str(fasta.description)
            if description:
                # Parse description
                parse = parse_description(description) # Ignore species name

                if parse is not None:
                    code, accession, start, end, species = parse
                    
                    if A_T == 'AT':
                        pair_no = code[2:]
                    else:
                        pair_no = code[1:]
    
                    # Append to DataFrame
                    df = pd.concat([df, pd.DataFrame({'type': [str(tat_type)], 'source': [str(source)], 'pair_no': [pair_no], 'accession': [str(accession)], \
                                                        'start': [int(start)],'end': [int(end)],'seq': [str(seq)]})], ignore_index=True)
    return df

In [ ]:
# Open each FASTA file and parse into pandas dataframe: at_exp_DF, T_exp_DF, AT_pre_DF, T_pre_DF
at_exp_df = fasta_to_df(at_exp_path, 'AT', 'exp', '2') # Experimental pairs imported from prior csv
t_exp_df = fasta_to_df(t_exp_path, 'T', 'exp','2')

In [ ]:
at_pre_df = fasta_to_df(at_pre_path, 'AT', 'pre', '2')
t_pre_df = fasta_to_df(t_pre_path, 'T', 'pre','2')

Pair Toxin-Antitoxin (TAT) pairs together

In [ ]:
# Combine toxin-antitoxin pairs, based on source, pair_no, accession
exp_paired_df = pd.merge(at_exp_df, t_exp_df, on=['type', 'source', 'pair_no', 'accession'], how='inner', suffixes=('_at','_t'))
pre_paired_df = pd.merge(at_pre_df, t_pre_df, on=['type', 'source', 'pair_no', 'accession'], how='inner', suffixes=('_at','_t'))

# Combine experimental + in silico predicted pairs
tat_paired_df = pd.concat([exp_paired_df, pre_paired_df], ignore_index=True)

In [ ]:
print(len(tat_paired_df['pair_no']))

Identify operon strand in genome: Forward strand, Reverse strand

In [ ]:
# Function
def strand_direction(row=pd.Series, A_T=str):
    if A_T == 'AT':
        # Identify antitoxin gene direction
        if row['start_at'] < row['end_at']:
            dir = 'F'
        elif row['start_at'] > row['end_at']:
            dir = 'R'

    elif A_T == 'T':
        # Identify toxin gene direction
        if row['start_t'] < row['end_t']:
            dir = 'F'
        elif row['start_t'] > row['end_t']:
            dir = 'R'
    
    return dir

# Identify strand direction: Forward vs. Reverse
tat_paired_df['dir_at'] = tat_paired_df.apply(strand_direction, axis=1, args=('AT',))
tat_paired_df['dir_t'] = tat_paired_df.apply(strand_direction, axis=1, args=('T',))

Find operon position in genome

In [ ]:
# Check if Toxin-Antitoxin pair always in same direction
mask = tat_paired_df['dir_at'] != tat_paired_df['dir_t']
print(tat_paired_df.index[mask].tolist()) # If empty, Toxin-Antitoxin pair always in same direction

# Exclude Toxin-Antitoxin pairs in different directions
tat_paired_df = tat_paired_df[tat_paired_df['dir_at'] == tat_paired_df['dir_t']]

# Take direction of Toxin gene as direction of operon
tat_paired_df['dir'] = tat_paired_df['dir_t']

In [ ]:
# Find operon position: Start, End, Length (of Operon)
tat_paired_df['start_operon'] = tat_paired_df.apply(lambda row: min(row['start_at'], row['start_t'], row['end_at'], row['end_t']), axis=1)
tat_paired_df['end_operon'] = tat_paired_df.apply(lambda row: max(row['start_at'], row['start_t'], row['end_at'], row['end_t']), axis=1)
tat_paired_df['len_operon'] = tat_paired_df['end_operon'] - tat_paired_df['start_operon'] + 1

'''
# Select max operon length (= context length for all operons)
max_len_operon = tat_paired_df['len_operon'].max()
if max_len_operon < MAX_SEQ_LENGTH:
    max_len = max_len_operon
else:
    max_len = MAX_SEQ_LENGTH
'''

### Find genomic regions

Identify unique genomes

In [ ]:
# Create new Pandas DataFrame with unique accession # as NCBI genome reference 
genome_df = pd.DataFrame(tat_paired_df['accession'].drop_duplicates())

In [ ]:
# Divide genome reference into 1k, 4k, 10k 
genome_df = genome_df.sample(frac=1, random_state=1).reset_index(drop=True) # Shuffle accession; Reset index, from 0

genome_df_1k = genome_df.iloc[0:train_size_1k] # Index rows 0-999
genome_df_5k = genome_df.iloc[train_size_1k:train_size_5k] # Index rows 1000-4999
genome_df_15k = genome_df.iloc[train_size_5k:train_size_15k] # Index rows 5000-14999

Fetch relevant genomes from NIH NCBI

In [ ]:
# Function
def NCBI_genome(accession=str):
    '''
    Access NCBI with email to return genome: Sequence
    '''
    Entrez.email = email_id
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", retmode="text")
    record = SeqIO.read(handle, "fasta")
    handle.close()
    return str(record.seq)

In [ ]:
# Find NCBI genome sequence for each accession no.
# genome_df_1k['seq_genome'] = genome_df_1k['accession'].apply(NCBI_genome)
genome_df_5k['seq_genome'] = genome_df_5k['accession'].apply(NCBI_genome)
# genome_df_15k['seq_genome'] = genome_df_15k['accession'].apply(NCBI_genome)

Combine genome dataframes

In [ ]:
# Import existing genome data
genome_df_exp = pd.read_csv(genome_csv_path_exp, index_col=0)
genome_df_1k = pd.read_csv(genome_csv_path_1k, index_col=0)
genome_df_5k = pd.read_csv(genome_csv_path_5k, index_col=0)

In [ ]:
genome_df_out = pd.merge(genome_df_exp, genome_df_1k, on=['accession', 'seq_genome'], how='outer')
genome_df_out = pd.merge(genome_df_out, genome_df_5k, on=['accession', 'seq_genome'], how='outer')
# genome_df = pd.merge(genome_df_out, genome_df_10k, on=['accession', 'seq_genome'], how='outer')

Export NCBI genome sequences as CSV file

In [ ]:
# Write genome_df as CSV file
genome_df_out.to_csv(genome_csv_path)

Import NCBI genome sequences from CSV file

In [ ]:
# genome_df = pd.read_csv(genome_csv_path, index_col=0)

Identify context length regions (+special token)

In [ ]:
# Assign genome to each TAT pair
tat_paired_df_out = pd.merge(tat_paired_df, genome_df_out, on=['accession'], how='inner')

In [ ]:
# Function
def assemble_seq_context(row=pd.Series):
    # Tokens
    token_type = TA_SPECIAL_TOKEN[row['type']]
    token_t = TA_SPECIAL_TOKEN['T']
    token_at = TA_SPECIAL_TOKEN['AT']

    # Case 1: Forward strand
    if row['dir'] == 'F':
        # Gene sequence
        gene_t = row['seq_genome'][row['start_t']:row['end_t']]
        gene_at = row['seq_genome'][row['start_at']:row['end_at']]

        # Case 1-A: Toxin - Antitoxin
        # Type token + T token + Toxin gene + T token + Spacer + AT token + Antitoxin gene + AT token
        if row['start_t'] < row['start_at']:
            # Spacer sequence
            spacer = row['seq_genome'][row['end_t']+1:row['start_at']-1]
            
            seq_context = token_type + token_t + gene_t + token_t + spacer + token_at + gene_at + token_at

        # Case 1-B: Antitoxin - Toxin
        # Type token + AT token + Antitoxin gene + Spacer + T token + Toxin gene
        elif row['start_t'] > row['start_at']:
            # Spacer sequence
            spacer = row['seq_genome'][row['end_at']+1:row['start_t']-1]
            
            seq_context = token_type + token_at + gene_at + token_at + spacer + token_t + gene_t + token_t

    # Case 2: Reverse strand
    elif row['dir'] == 'R':
        # Gene sequence
        gene_t = row['seq_genome'][row['end_t']:row['start_t']]
        gene_at = row['seq_genome'][row['end_at']:row['start_at']]

        # Case 2-A: Toxin - Antitoxin
        # AT token + Antitoxin gene + AT token + Spacer + T token + Toxin gene + T token + Type token
        if row['end_at'] < row['end_t']:
            # Spacer sequence
            spacer = row['seq_genome'][row['start_at']+1:row['end_t']-1]

            seq_context = token_at + gene_at + token_at + spacer + token_t + gene_t + token_t + token_type

        # Case 2-B: Antitoxin - Toxin
        # T token + Toxin gene + T token + Spacer + AT token + Antitoxin gene + AT token + Type token
        elif row['end_at'] > row['end_t']:
            # Spacer sequence
            spacer = row['seq_genome'][row['start_t']+1:row['end_at']-1]
        
            seq_context = token_t + gene_t + token_t + spacer + token_at + gene_at + token_at + token_type

    # Add spaces if context length is shorter than max_len (eg when genome length is short)
    global MAX_CONTEXT_LENGTH
    if len(seq_context) < MAX_CONTEXT_LENGTH:
        num_spaces = MAX_CONTEXT_LENGTH - len(seq_context)
        padding_3 = ' ' * num_spaces
    else:
        padding_3 = ''

    seq_context += padding_3

    return seq_context           

def assemble_seq_context_onetok(row=pd.Series):
    # Tokens
    token_type = TA_SPECIAL_TOKEN[row['type']]
    token_t = TA_SPECIAL_TOKEN['T']
    token_at = TA_SPECIAL_TOKEN['AT']

    # Case 1: Forward strand
    if row['dir'] == 'F':
        # Gene sequence
        gene_t = row['seq_genome'][row['start_t']:row['end_t']]
        gene_at = row['seq_genome'][row['start_at']:row['end_at']]

        # Case 1-A: Toxin - Antitoxin
        # Type token + T token + Toxin gene + T token + Spacer + AT token + Antitoxin gene + AT token
        if row['start_t'] < row['start_at']:
            # Spacer sequence
            spacer = row['seq_genome'][row['end_t']+1:row['start_at']-1]
            
            seq_context = token_type + token_t + gene_t + spacer + token_at + gene_at

        # Case 1-B: Antitoxin - Toxin
        # Type token + AT token + Antitoxin gene + Spacer + T token + Toxin gene
        elif row['start_t'] > row['start_at']:
            # Spacer sequence
            spacer = row['seq_genome'][row['end_at']+1:row['start_t']-1]
            
            seq_context = token_type + token_at + gene_at + spacer + token_t + gene_t

    # Case 2: Reverse strand
    elif row['dir'] == 'R':
        # Gene sequence
        gene_t = row['seq_genome'][row['end_t']:row['start_t']]
        gene_at = row['seq_genome'][row['end_at']:row['start_at']]

        # Case 2-A: Toxin - Antitoxin
        # AT token + Antitoxin gene + AT token + Spacer + T token + Toxin gene + T token + Type token
        if row['end_at'] < row['end_t']:
            # Spacer sequence
            spacer = row['seq_genome'][row['start_at']+1:row['end_t']-1]

            seq_context =  gene_at + token_at + spacer + gene_t + token_t + token_type

        # Case 2-B: Antitoxin - Toxin
        # T token + Toxin gene + T token + Spacer + AT token + Antitoxin gene + AT token + Type token
        elif row['end_at'] > row['end_t']:
            # Spacer sequence
            spacer = row['seq_genome'][row['start_t']+1:row['end_at']-1]
        
            seq_context =  gene_t + token_t + spacer + gene_at + token_at + token_type

    # Add spaces if context length is shorter than max_len (eg when genome length is short)
    global MAX_CONTEXT_LENGTH
    if len(seq_context) < MAX_CONTEXT_LENGTH:
        num_spaces = MAX_CONTEXT_LENGTH - len(seq_context)
        padding_3 = ' ' * num_spaces
    else:
        padding_3 = ''

    seq_context += padding_3

    return seq_context                                          

In [ ]:
# Add padding + operon + special token
# tat_paired_df_out['seq_context'] = tat_paired_df_out.apply(assemble_seq_context, axis=1)
tat_paired_df_out['seq_context'] = tat_paired_df_out.apply(assemble_seq_context_onetok, axis=1)

Export as CSV file

In [ ]:
# tat_paired_df.to_csv(csv_output)

### Export as JSON file

In [ ]:
# Functions
def write_json(data, OUTPUT_PATH):
    with open(OUTPUT_PATH, 'w') as file:
        json.dump(data, file)

def convert_df_to_list(df=pd.DataFrame):
    records = []
    global MAX_CONTEXT_LENGTH

    for index, row in df.iterrows():
        # Append rows with context length less than or equal to max context length
        if len(row['seq_context']) <= MAX_CONTEXT_LENGTH:
            records.append({
                'record': row['pair_no'],
                'text': row['seq_context']
                })

    return records

# Convert Pandas DataFrame into list
data_list = convert_df_to_list(tat_paired_df_out)
print('Num TA pairs: ' + str(len(data_list))) # Number of TA pairs in training data

# Write JSON file from list
write_json(data_list, json_output_path)

### Miscellaneous

In [ ]:
'''
# From Evo CRISPR training
def parse_fasta_with_biopython(fname, id_to_cas_id):
    records = []

    if fname.endswith('.gz'):
        file_ = gzip.open(fname, 'rt')
    else:
        file_ = open(fname, 'r')

    for record in SeqIO.parse(file_, 'fasta'):
        seq = str(record.seq)

        if 'NNN' in seq:
            continue

        if len(seq) >= MAX_seq_LENGTH - 2: # Minus start and end tokens.
            extra = len(seq) - (MAX_seq_LENGTH - 2)
            seq = seq[:-extra]

        cas_id = id_to_cas_id[record.id]

        seq = CAS_ID_TO_START_TOKEN[cas_id] + seq # Encode start token.
        # Stop token (EOD) is appended in downstream preprocess_data.py script.

        records.append({
            'record': record.id,
            'text': seq,
        })

    file_.close()

    return records


def write_json(fname, data, output_path):
    with open(output_path, 'at') as f:
        for record in data:
            f.write(json.dumps(record) + '\n')


def process_file(fname, id_to_cas_id, output_file):
    parsed_data = parse_fasta_with_biopython(fname, id_to_cas_id)
    write_json(fname, parsed_data, output_file)


if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Parse Cas sequences and output JSONL.")
    parser.add_argument("fasta_path", type=str,
                        help="Path to directory containing input FASTA file.")
    parser.add_argument("metadata_path", type=str,
                        help="Path to Cas metadata file.")
    parser.add_argument("output_path", type=str,
                        help="Path to output JSON file.")
    args = parser.parse_args()

    id_to_cas_id = load_metadata(args.metadata_path)

    process_file(args.fasta_path, id_to_cas_id, args.output_path)
'''